# 🎵 Spotify Wrapped - Extensive EDA & Data Visualization

This notebook provides comprehensive exploratory data analysis of your Spotify listening data.

## Table of Contents
1. [Setup & Data Loading](#1-setup--data-loading)
2. [Data Overview](#2-data-overview)
3. [Listening Patterns](#3-listening-patterns)
4. [Audio Features Analysis](#4-audio-features-analysis)
5. [Genre Analysis](#5-genre-analysis)
6. [Artist Analysis](#6-artist-analysis)
7. [Mood & Vibe Analysis](#7-mood--vibe-analysis)
8. [Time-Based Analysis](#8-time-based-analysis)
9. [Correlation Analysis](#9-correlation-analysis)
10. [User Profile Summary](#10-user-profile-summary)

---
## 1. Setup & Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('dark_background')
sns.set_palette('husl')

# Spotify colors
SPOTIFY_GREEN = '#1DB954'
SPOTIFY_BLACK = '#191414'
COLORS = ['#1DB954', '#1ed760', '#4ECDC4', '#FFE66D', '#FF6B6B', '#a855f7', '#ec4899']

print('✅ Libraries loaded successfully!')

In [ ]:
# Load the data
df = pd.read_csv('../data/SytheticData1000.csv')

# Normalize column names
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

print(f'📊 Loaded {len(df):,} tracks')
print(f'📅 Columns: {len(df.columns)}')

---
## 2. Data Overview

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Data types and info
df.info()

In [ ]:
# Statistical summary
df.describe()

In [ ]:
# Check for missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing': missing, 'Percentage': missing_pct})
missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)

In [ ]:
# Quick stats
print('=' * 50)
print('📊 QUICK STATISTICS')
print('=' * 50)
print(f"Total Tracks: {len(df):,}")
print(f"Unique Artists: {df['artist_name(s)'].nunique():,}")
print(f"Unique Albums: {df['album_name'].nunique():,}")
print(f"Total Duration: {df['duration_(ms)'].sum() / (1000*60*60):.1f} hours")
print(f"Avg Track Duration: {df['duration_(ms)'].mean() / (1000*60):.2f} minutes")
print(f"Explicit Tracks: {df['explicit'].sum()} ({df['explicit'].mean()*100:.1f}%)")

---
## 3. Listening Patterns

In [ ]:
# Parse dates
df['added_at'] = pd.to_datetime(df['added_at'], errors='coerce')
df['added_date'] = df['added_at'].dt.date
df['added_hour'] = df['added_at'].dt.hour
df['added_day'] = df['added_at'].dt.dayofweek
df['added_month'] = df['added_at'].dt.month
df['added_year'] = df['added_at'].dt.year

print('✅ Date features extracted')

In [ ]:
# Tracks added over time
daily = df.groupby('added_date').size().reset_index(name='tracks')
daily['added_date'] = pd.to_datetime(daily['added_date'])
daily['rolling_avg'] = daily['tracks'].rolling(7, min_periods=1).mean()

fig = go.Figure()
fig.add_trace(go.Bar(x=daily['added_date'], y=daily['tracks'], name='Daily', 
                     marker_color='rgba(29, 185, 84, 0.4)'))
fig.add_trace(go.Scatter(x=daily['added_date'], y=daily['rolling_avg'], 
                         name='7-Day Avg', line=dict(color=SPOTIFY_GREEN, width=3)))
fig.update_layout(title='📅 Tracks Added Over Time', template='plotly_dark', height=400)
fig.show()

In [ ]:
# Hour vs Day Heatmap
heatmap_data = df.groupby(['added_day', 'added_hour']).size().unstack(fill_value=0)
day_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=list(range(24)),
    y=day_labels,
    colorscale=[[0, '#191414'], [0.5, '#1DB954'], [1, '#1ed760']]
))
fig.update_layout(title='⏰ When You Add Music (Hour vs Day)', 
                  xaxis_title='Hour', yaxis_title='Day', template='plotly_dark', height=400)
fig.show()

In [ ]:
# Monthly trends
monthly = df.groupby([df['added_at'].dt.to_period('M')]).size().reset_index(name='tracks')
monthly['period'] = monthly['added_at'].astype(str)

fig = px.bar(monthly, x='period', y='tracks', title='📆 Monthly Track Additions',
             color='tracks', color_continuous_scale=['#191414', '#1DB954'])
fig.update_layout(template='plotly_dark', height=400)
fig.show()

---
## 4. Audio Features Analysis

In [ ]:
# Audio features
audio_features = ['danceability', 'energy', 'speechiness', 'acousticness', 
                  'instrumentalness', 'liveness', 'valence']

# Distribution plots
fig = make_subplots(rows=2, cols=4, subplot_titles=audio_features + ['tempo'])

for i, feature in enumerate(audio_features):
    row = i // 4 + 1
    col = i % 4 + 1
    fig.add_trace(go.Histogram(x=df[feature], name=feature, 
                               marker_color=COLORS[i % len(COLORS)], nbinsx=30), row=row, col=col)

# Add tempo
fig.add_trace(go.Histogram(x=df['tempo'], name='tempo', marker_color=COLORS[0], nbinsx=30), row=2, col=4)

fig.update_layout(title='📊 Audio Features Distribution', template='plotly_dark', 
                  height=600, showlegend=False)
fig.show()

In [ ]:
# Radar chart of average audio features
avg_features = df[audio_features].mean()

fig = go.Figure(data=go.Scatterpolar(
    r=avg_features.values.tolist() + [avg_features.values[0]],
    theta=audio_features + [audio_features[0]],
    fill='toself',
    line_color=SPOTIFY_GREEN,
    fillcolor='rgba(29, 185, 84, 0.3)'
))
fig.update_layout(title='🎭 Your Audio DNA (Average Features)', template='plotly_dark',
                  polar=dict(radialaxis=dict(visible=True, range=[0, 1])), height=500)
fig.show()

In [ ]:
# Box plots for audio features
fig = go.Figure()
for i, feature in enumerate(audio_features):
    fig.add_trace(go.Box(y=df[feature], name=feature, marker_color=COLORS[i % len(COLORS)]))

fig.update_layout(title='📦 Audio Features Box Plot', template='plotly_dark', height=400)
fig.show()

In [ ]:
# Tempo distribution with genres
fig = go.Figure()
fig.add_trace(go.Histogram(x=df['tempo'], nbinsx=50, marker_color=SPOTIFY_GREEN, opacity=0.8))
fig.add_vline(x=df['tempo'].mean(), line_dash='dash', line_color='#FFE66D',
              annotation_text=f"Avg: {df['tempo'].mean():.0f} BPM")
fig.update_layout(title='🥁 Tempo Distribution', xaxis_title='BPM', template='plotly_dark', height=400)
fig.show()

---
## 5. Genre Analysis

In [ ]:
# Parse genres
def parse_genres(genre_str):
    if pd.isna(genre_str) or genre_str == '':
        return []
    return [g.strip() for g in str(genre_str).split(',')]

df['genres_list'] = df['genres'].apply(parse_genres)
df['primary_genre'] = df['genres_list'].apply(lambda x: x[0] if len(x) > 0 else 'unknown')
df['genre_count'] = df['genres_list'].apply(len)

# Get all genres
all_genres = [g for genres in df['genres_list'] for g in genres]
genre_counts = pd.Series(all_genres).value_counts()

print(f'📊 Total unique genres: {len(genre_counts)}')
print(f'📊 Average genres per track: {df["genre_count"].mean():.1f}')

In [ ]:
# Top 20 genres bar chart
top_genres = genre_counts.head(20)

fig = go.Figure(go.Bar(
    x=top_genres.values,
    y=top_genres.index,
    orientation='h',
    marker=dict(color=list(range(len(top_genres))), 
                colorscale=[[0, '#1DB954'], [0.5, '#4ECDC4'], [1, '#FF6B6B']])
))
fig.update_layout(title='🎸 Top 20 Genres', yaxis=dict(autorange='reversed'),
                  template='plotly_dark', height=600)
fig.show()

In [ ]:
# Genre Sunburst
top_15 = dict(genre_counts.head(15))
other = genre_counts[15:].sum()
if other > 0:
    top_15['Other'] = other

fig = go.Figure(go.Sunburst(
    labels=list(top_15.keys()),
    values=list(top_15.values()),
    marker=dict(colors=COLORS * 3)
))
fig.update_layout(title='🎸 Genre Universe', template='plotly_dark', height=500)
fig.show()

In [ ]:
# Genre treemap
genre_df = pd.DataFrame({'genre': top_genres.index, 'count': top_genres.values})

fig = px.treemap(genre_df, path=['genre'], values='count', 
                 color='count', color_continuous_scale=['#191414', '#1DB954'])
fig.update_layout(title='🎸 Genre Treemap', template='plotly_dark', height=500)
fig.show()

---
## 6. Artist Analysis

In [ ]:
# Top artists
artist_counts = df['artist_name(s)'].value_counts()
top_artists = artist_counts.head(20)

print(f'📊 Unique artists: {len(artist_counts)}')
print(f'🥇 Top artist: {top_artists.index[0]} ({top_artists.values[0]} tracks)')

In [ ]:
# Top 15 artists bar chart
fig = go.Figure(go.Bar(
    x=top_artists.head(15).values,
    y=top_artists.head(15).index,
    orientation='h',
    marker=dict(color=top_artists.head(15).values,
                colorscale=[[0, '#4ECDC4'], [0.5, '#1DB954'], [1, '#1ed760']])
))
fig.update_layout(title='🎤 Top 15 Artists', yaxis=dict(autorange='reversed'),
                  template='plotly_dark', height=500)
fig.show()

In [ ]:
# Artist stats
artist_stats = df.groupby('artist_name(s)').agg({
    'track_uri': 'count',
    'popularity': 'mean',
    'energy': 'mean',
    'valence': 'mean',
    'danceability': 'mean'
}).rename(columns={'track_uri': 'track_count'})

# Top 10 artists detailed
top_10 = artist_stats.nlargest(10, 'track_count')
top_10.round(2)

In [ ]:
# Artist energy vs valence scatter
top_20_artists = artist_stats.nlargest(20, 'track_count').reset_index()

fig = px.scatter(top_20_artists, x='energy', y='valence', size='track_count',
                 color='track_count', hover_name='artist_name(s)',
                 color_continuous_scale=['#4ECDC4', '#1DB954'],
                 title='🎤 Top 20 Artists: Energy vs Valence')
fig.update_layout(template='plotly_dark', height=500)
fig.show()

---
## 7. Mood & Vibe Analysis

In [ ]:
# Mood quadrants based on energy and valence
def get_mood(row):
    if row['energy'] > 0.5 and row['valence'] > 0.5:
        return 'Happy & Energetic 🔥'
    elif row['energy'] > 0.5 and row['valence'] <= 0.5:
        return 'Angry & Intense ⚡'
    elif row['energy'] <= 0.5 and row['valence'] > 0.5:
        return 'Peaceful & Happy 😌'
    else:
        return 'Sad & Calm 🌙'

df['mood'] = df.apply(get_mood, axis=1)

# Mood distribution
mood_counts = df['mood'].value_counts()
print('🎭 MOOD DISTRIBUTION')
for mood, count in mood_counts.items():
    print(f"  {mood}: {count} tracks ({count/len(df)*100:.1f}%)")

In [ ]:
# Mood quadrant scatter
df_sample = df.sample(min(1000, len(df)), random_state=42)

color_map = {
    'Happy & Energetic 🔥': '#1DB954',
    'Angry & Intense ⚡': '#FF6B6B',
    'Peaceful & Happy 😌': '#FFE66D',
    'Sad & Calm 🌙': '#4ECDC4'
}

fig = px.scatter(df_sample, x='energy', y='valence', color='mood',
                 color_discrete_map=color_map, hover_data=['track_name', 'artist_name(s)'],
                 title='🎭 Music Mood Map (Energy x Valence)')
fig.add_hline(y=0.5, line_dash='dash', line_color='white', opacity=0.3)
fig.add_vline(x=0.5, line_dash='dash', line_color='white', opacity=0.3)
fig.update_layout(template='plotly_dark', height=600)
fig.show()

In [ ]:
# Mood pie chart
fig = go.Figure(go.Pie(
    labels=mood_counts.index,
    values=mood_counts.values,
    marker=dict(colors=[color_map[m] for m in mood_counts.index]),
    hole=0.4
))
fig.update_layout(title='🎭 Mood Distribution', template='plotly_dark', height=450)
fig.show()

In [ ]:
# 3D scatter: Energy, Valence, Danceability
df_sample = df.sample(min(500, len(df)), random_state=42)

fig = px.scatter_3d(df_sample, x='energy', y='valence', z='danceability',
                    color='mood', color_discrete_map=color_map,
                    hover_data=['track_name', 'artist_name(s)'],
                    title='🎵 3D Mood Space')
fig.update_layout(template='plotly_dark', height=600)
fig.show()

---
## 8. Time-Based Analysis

In [ ]:
# Energy over time
monthly_features = df.groupby(df['added_at'].dt.to_period('M')).agg({
    'energy': 'mean',
    'valence': 'mean',
    'danceability': 'mean',
    'tempo': 'mean',
    'track_uri': 'count'
}).rename(columns={'track_uri': 'track_count'})
monthly_features.index = monthly_features.index.astype(str)

fig = make_subplots(rows=2, cols=1, subplot_titles=['Track Count', 'Energy & Valence Trends'])

fig.add_trace(go.Bar(x=monthly_features.index, y=monthly_features['track_count'],
                     marker_color=SPOTIFY_GREEN, name='Tracks'), row=1, col=1)
fig.add_trace(go.Scatter(x=monthly_features.index, y=monthly_features['energy'],
                         mode='lines+markers', name='Energy', line=dict(color='#FF6B6B')), row=2, col=1)
fig.add_trace(go.Scatter(x=monthly_features.index, y=monthly_features['valence'],
                         mode='lines+markers', name='Valence', line=dict(color='#4ECDC4')), row=2, col=1)

fig.update_layout(title='📅 Monthly Trends', template='plotly_dark', height=600)
fig.show()

In [ ]:
# Release year distribution
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year

year_counts = df['release_year'].value_counts().sort_index()
year_counts = year_counts[year_counts.index >= 1980]  # Filter realistic years

fig = go.Figure(go.Bar(x=year_counts.index, y=year_counts.values, marker_color=SPOTIFY_GREEN))
fig.update_layout(title='📀 Release Year Distribution', xaxis_title='Year',
                  template='plotly_dark', height=400)
fig.show()

In [ ]:
# Weekend vs Weekday
df['is_weekend'] = df['added_day'].isin([5, 6])
weekend_counts = df['is_weekend'].value_counts()

fig = go.Figure(go.Pie(
    labels=['Weekday', 'Weekend'],
    values=[weekend_counts.get(False, 0), weekend_counts.get(True, 0)],
    marker=dict(colors=[SPOTIFY_GREEN, '#4ECDC4']),
    hole=0.5
))
fig.update_layout(title='📆 Weekend vs Weekday Additions', template='plotly_dark', height=400)
fig.show()

---
## 9. Correlation Analysis

In [ ]:
# Correlation heatmap
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'tempo', 'popularity']
corr = df[features].corr()

fig = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=features,
    y=features,
    colorscale=[[0, '#FF6B6B'], [0.5, '#191414'], [1, '#1DB954']],
    zmid=0
))
fig.update_layout(title='🔗 Feature Correlation Matrix', template='plotly_dark', height=600)
fig.show()

In [ ]:
# Pairplot for key features (using plotly)
key_features = ['energy', 'valence', 'danceability', 'acousticness']

fig = px.scatter_matrix(df.sample(500), dimensions=key_features, color='mood',
                        color_discrete_map=color_map, title='🔗 Feature Relationships')
fig.update_layout(template='plotly_dark', height=800)
fig.show()

In [ ]:
# Popularity vs Audio Features
fig = make_subplots(rows=2, cols=2, subplot_titles=['Popularity vs Energy', 'Popularity vs Danceability',
                                                     'Popularity vs Valence', 'Popularity vs Acousticness'])

sample = df.sample(500)
fig.add_trace(go.Scatter(x=sample['energy'], y=sample['popularity'], mode='markers',
                         marker=dict(color=SPOTIFY_GREEN, opacity=0.5)), row=1, col=1)
fig.add_trace(go.Scatter(x=sample['danceability'], y=sample['popularity'], mode='markers',
                         marker=dict(color='#4ECDC4', opacity=0.5)), row=1, col=2)
fig.add_trace(go.Scatter(x=sample['valence'], y=sample['popularity'], mode='markers',
                         marker=dict(color='#FFE66D', opacity=0.5)), row=2, col=1)
fig.add_trace(go.Scatter(x=sample['acousticness'], y=sample['popularity'], mode='markers',
                         marker=dict(color='#FF6B6B', opacity=0.5)), row=2, col=2)

fig.update_layout(title='📊 Popularity vs Audio Features', template='plotly_dark',
                  height=600, showlegend=False)
fig.show()

---
## 10. User Profile Summary

In [ ]:
# Generate comprehensive user profile
print('=' * 60)
print('🎵 YOUR SPOTIFY PROFILE SUMMARY')
print('=' * 60)

# Basic stats
print(f"\n📊 LISTENING STATS")
print(f"   Total Tracks: {len(df):,}")
print(f"   Total Artists: {df['artist_name(s)'].nunique():,}")
print(f"   Total Albums: {df['album_name'].nunique():,}")
print(f"   Total Hours: {df['duration_(ms)'].sum() / (1000*60*60):.1f}")

# Top content
print(f"\n🎤 TOP ARTIST: {artist_counts.index[0]} ({artist_counts.values[0]} tracks)")
print(f"🎸 TOP GENRE: {genre_counts.index[0]} ({genre_counts.values[0]} tracks)")

# Audio profile
print(f"\n🎭 AUDIO PROFILE")
print(f"   Avg Energy: {df['energy'].mean():.2f}")
print(f"   Avg Valence: {df['valence'].mean():.2f}")
print(f"   Avg Danceability: {df['danceability'].mean():.2f}")
print(f"   Avg Tempo: {df['tempo'].mean():.0f} BPM")

# Mood
dominant_mood = mood_counts.index[0]
print(f"\n🎭 DOMINANT MOOD: {dominant_mood}")

# Listening patterns
night_ratio = ((df['added_hour'] >= 22) | (df['added_hour'] < 6)).mean()
weekend_ratio = df['is_weekend'].mean()
print(f"\n⏰ LISTENING PATTERNS")
print(f"   Night Owl Score: {night_ratio*100:.1f}%")
print(f"   Weekend Listener: {weekend_ratio*100:.1f}%")

print('\n' + '=' * 60)

In [ ]:
# Final dashboard summary
fig = make_subplots(
    rows=2, cols=3,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'pie'}, {'type': 'bar'}, {'type': 'scatterpolar'}]],
    subplot_titles=['', '', '', 'Mood Distribution', 'Top 5 Artists', 'Audio Profile']
)

# KPIs
fig.add_trace(go.Indicator(mode='number', value=len(df), title={'text': 'Tracks'}), row=1, col=1)
fig.add_trace(go.Indicator(mode='number', value=df['duration_(ms)'].sum()/(1000*60*60), 
                           title={'text': 'Hours'}, number={'valueformat': '.0f'}), row=1, col=2)
fig.add_trace(go.Indicator(mode='number', value=df['artist_name(s)'].nunique(), 
                           title={'text': 'Artists'}), row=1, col=3)

# Mood pie
fig.add_trace(go.Pie(labels=mood_counts.head(4).index, values=mood_counts.head(4).values,
                     marker=dict(colors=COLORS[:4]), hole=0.4), row=2, col=1)

# Top artists bar
fig.add_trace(go.Bar(x=top_artists.head(5).values, y=top_artists.head(5).index,
                     orientation='h', marker_color=SPOTIFY_GREEN), row=2, col=2)

# Audio radar
fig.add_trace(go.Scatterpolar(
    r=avg_features.values.tolist() + [avg_features.values[0]],
    theta=audio_features + [audio_features[0]],
    fill='toself', line_color=SPOTIFY_GREEN
), row=2, col=3)

fig.update_layout(title='📊 Your Spotify Wrapped Dashboard', template='plotly_dark',
                  height=700, showlegend=False)
fig.show()

---
## 🎉 Analysis Complete!

This notebook has covered:
- ✅ Data overview and statistics
- ✅ Listening patterns over time
- ✅ Audio features analysis
- ✅ Genre and artist breakdown
- ✅ Mood and vibe classification
- ✅ Temporal trends
- ✅ Feature correlations
- ✅ Complete user profile

Run the Streamlit app for an interactive experience:
```bash
streamlit run app.py
```